## Automate ML experiments with Skorch and W&B
Checkout [this report](https://app.wandb.ai/cayush/uncategorized/reports/Automate-ML-experiments-with-skorch-and-W%26B--Vmlldzo4NjA4Ng/edit) to follow along with the code

In [2]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torchvision import datasets,transforms
from torch.utils.data import Dataset,DataLoader
import numpy as np 
import pandas as pd 

import os
print(os.listdir())
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt


['.ipynb_checkpoints', 'wandb', 'test.csv', 'train.csv', 'Untitled.ipynb', 'sampleSubmission.csv']


In [3]:

xy = np.loadtxt('train.csv',delimiter=',',skiprows = 1, usecols = np.arange(1,94))
X = pd.read_csv('train.csv', sep = ',')
Y =  X['target'].map({'Class_1': 1, 'Class_2': 2,
                                  'Class_3': 3, 'Class_4': 4,
                                  'Class_5': 5, 'Class_6': 6,
                                  'Class_7': 7, 'Class_8': 8,
                                  'Class_9': 9})

Y = Y.astype('float64')
X =X.drop(['id','target'],axis=1)

# Hyper-parameter Sweep

In [4]:
import wandb
sweep_config = {
    'method': 'random', #grid, random
    'metric': {
      'name': 'valid_loss',
      'goal': 'minimize'   
    },
    'parameters': {

        'learning_rate': {
            'values': [0.1, 0.01,0.001]
        },

        'fc_layer1':{
              'values':[93]
        },
        'fc_layer2':{
            'values':[8,10,12,16]
        },
        'fc_layer3':{
            'values':[4,8,10,20]
        }
    }
}

config_defaults = {
        'learning_rate': 0.001,

        'fc_layer1' : 93,
        'fc_layer2' : 10,
        'fc_layer3' : 10,
    }


wandb: Appending key for api.wandb.ai to your netrc file: /home/ayush/.netrc
Successfully logged in to Weights & Biases!


In [5]:
import skorch
from skorch import NeuralNetClassifier

'''
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.l1 = nn.Linear(93, 16)
        self.l2 = nn.Linear(16,8)
        self.l3 = nn.Linear(8,1)
    def forward(self,x):
        x = F.relu(self.l1(x))
        x = F.relu(self.l2(x))
        return self.l3(x)
'''

class parseModel(nn.Module):
    def __init__(self,def_config={}):
        super(parseModel,self).__init__()
        self.layers = torch.nn.ModuleList();
        self.layer_dims = []
        config_dict = {str(k):v for k, v in dict(def_config).items()}
        for key in config_dict:
            if key.find('fc_layer') != -1:
                self.layer_dims.append(config_dict[key])
        for i in range(len(self.layer_dims)-1):
            self.layers.append(torch.nn.Linear(self.layer_dims[i],self.layer_dims[i+1]))
        self.layers.append(torch.nn.Linear(self.layer_dims[len(self.layer_dims)-1],1))
    
    def print_model(self):
        print(self.layers)
        
    def forward(self,x):
        y = x
        for i in range(len(self.layers)):
            y = self.layers[i](y)
        return y

class ClassifierNet():
    def __init__(self,config={}):
        self.net = None
        self.config = config
        self.data_loaded = False

        
    def loadData(self,X,Y,split=True,test=0.2):
        X_train,X_test,y_train,y_test = train_test_split(X.values.astype('float32'),
                                             Y.values.astype('float32').reshape(-1,1),
                                             test_size=.2)

        self.X_train = X_train
        self.y_train = y_train
        self.X_test = X_test
        self.y_test = y_test
        
        self.data_loaded = True
        
    def buildNetwork(self,new_config):
        self.config = new_config
        model = parseModel(self.config)
        self.net = model
        
    def train(self):
        if self.data_loaded == False:
            print('Data not loaded')
            return
        wandb_run = wandb.init()
        config = wandb.config
        self.buildNetwork(config)
        
        from skorch.dataset import Dataset
        from skorch.helper import predefined_split
        from skorch.callbacks import WandbLogger

        valid_ds = Dataset(self.X_test, self.y_test)


        optimizer = optim.Adam
        Net = NeuralNetClassifier(
        self.net,
        criterion= nn.BCEWithLogitsLoss ,
        max_epochs=15,
        optimizer=optimizer,
        optimizer__lr = self.config.learning_rate,
        callbacks=[WandbLogger(wandb_run)],
        train_split=predefined_split(valid_ds)

        )
        Net.fit(self.X_train, self.y_train)
    
    def runSweep(self,sweep_config):
        sweep_id = wandb.sweep(sweep_config)
        wandb.agent(sweep_id,function=self.train)



In [6]:
classifier  = ClassifierNet()
classifier.loadData(X,Y)

In [ ]:
classifier.runSweep(sweep_config)

Create sweep with ID: wmrcdkdu
Sweep URL: https://app.wandb.ai/cayush/uncategorized/sweeps/wmrcdkdu
wandb: Agent Starting Run: dx1sfl5x with config:
	fc_layer1: 93
	fc_layer2: 8
	fc_layer3: 10
	learning_rate: 0.1
wandb: Agent Started Run: dx1sfl5x


wandb: Wandb version 0.8.32 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


Layers:
 {'fc_layer1': 93, 'fc_layer2': 8, 'fc_layer3': 10, 'learning_rate': 0.1}
  epoch        train_loss    valid_acc        valid_loss     dur
-------  ----------------  -----------  ----------------  ------
      1  -1816534302.0274       0.0000  -8099537162.7563  1.9372
      2  -31726719229.3625       0.0000  -69303167321.5255  1.7095
      3  -135743356720.1467       0.0000  -224680946745.5876  1.8949
      4  -346740250671.2469       0.0000  -500873832968.2741  1.8314
      5  -686419442680.6771       0.0000  -915889496577.6548  1.6125
      6  -1169829847863.1387       0.0000  -1482626177065.0396  1.8566
      7  -1808126175820.8899       0.0000  -2211041964018.7612  1.8216
      8  -2610263512208.5127       0.0000  -3109475588560.6724  1.8287
      9  -3584023634064.6367       0.0000  -4185442274779.9248  1.8241
     10  -4736609506228.7441       0.0000  -5446077907706.5391  1.8216
     11  -6075012743660.4932       0.0000  -6898425630161.3340  1.8217
     12  -7606189872692

wandb: Wandb version 0.8.32 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


Layers:
 {'fc_layer1': 93, 'fc_layer2': 16, 'fc_layer3': 8, 'learning_rate': 0.01}
  epoch     train_loss    valid_acc      valid_loss     dur
-------  -------------  -----------  --------------  ------
      1  -2948283.6087       0.0000  -13057511.7847  1.7524
      2  -50906239.0956       0.0000  -111075490.0065  2.2438
      3  -217419626.3559       0.0000  -359767943.8811  1.9029
      4  -555104995.5929       0.0000  -801773284.1577  2.1777
      5  -1098692126.6257       0.0000  -1465905849.7531  1.9941
      6  -1872260596.9950       0.0000  -2372805586.3271  1.9550
      7  -2893660256.1280       0.0000  -3538405109.9056  2.0597
      8  -4177217730.0764       0.0000  -4976044647.9224  1.7868
      9  -5735375884.7426       0.0000  -6697738171.1597  1.8939
     10  -7579666180.9129       0.0000  -8714915009.2825  2.0734
     11  -9721265978.4484       0.0000  -11038834223.3277  1.9877
     12  -12171309282.4293       0.0000  -13680816329.8875  2.3482
     13  -14941029847.0623

...

Layers:
 {'fc_layer1': 93, 'fc_layer2': 16, 'fc_layer3': 4, 'learning_rate': 0.1}
  epoch        train_loss    valid_acc        valid_loss     dur
-------  ----------------  -----------  ----------------  ------
      1  -1453579289.7088       0.0000  -6480450403.4544  2.6494
      2  -25382511727.9114       0.0000  -55444011803.6354  2.5732
      3  -108596392197.9679       0.0000  -179746840189.4350  2.3976
      4  -277394486586.3036       0.0000  -400701800034.9580  2.4481
      5  -549138241162.9067       0.0000  -732714546144.8894  2.8625
      6  -935866545542.4282       0.0000  -1186104002704.3000  2.0732
      7  -1446503874420.9897       0.0000  -1768837166366.6140  2.1544
      8  -2088214152463.4421       0.0000  -2487584558735.9688  1.8932
      9  -2867222473888.6479       0.0000  -3348358175311.7622  1.9611
     10  -3789291178389.8599       0.0000  -4356867074716.5454  1.8416
     11  -4860014413604.9766       0.0000  -5518746728684.9697  1.8644
     12  -6084957087619.

wandb: Wandb version 0.8.32 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


Layers:
 {'fc_layer1': 93, 'fc_layer2': 16, 'fc_layer3': 10, 'learning_rate': 0.1}
  epoch        train_loss    valid_acc         valid_loss     dur
-------  ----------------  -----------  -----------------  ------
      1  -3629864814.2202       0.0000  -16191744303.1622  2.0103
      2  -63443378289.5249       0.0000  -138593333340.6697  2.3047
      3  -271471852485.7068       0.0000  -449344089410.3581  2.5204
      4  -693461026331.5331       0.0000  -1001724543208.9980  1.9312
      5  -1372814159979.7329       0.0000  -1831749662766.3350  2.1377
      6  -2339629049121.6460       0.0000  -2965216590029.1973  2.6933
      7  -3616215608354.2148       0.0000  -4422041141459.8164  2.2326
      8  -5220483907485.3691       0.0000  -6218902111966.0762  2.1324
      9  -7167998687532.4443       0.0000  -8370829375619.0615  2.5903
     10  -9473165515805.4160       0.0000  -10892094707223.4980  2.4210
     11  -12149964369460.4395       0.0000  -13796779713142.1543  2.4416
     12  -15

wandb: Wandb version 0.8.32 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


Layers:
 {'fc_layer1': 93, 'fc_layer2': 8, 'fc_layer3': 8, 'learning_rate': 0.001}
  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1    -1742.2470       0.0000    -7241.9362  2.3818
      2   -26741.2106       0.0000   -57484.6632  2.5821
      3  -111356.6430       0.0000  -183338.0826  2.3894
